In [9]:
import re
from collections import Counter

In [10]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    "return (known([word]) or known(reductions(word)) or [word])"
    return (known(Priority_1(word)) or known(Priority_2(word)) or known(edits1(word)) or  known(edits2(word)) or [word] or known([word]))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def Priority_1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    "Priority 1: single letter -> doubling letters (ex. aple -> apple)"
    doubling   = [L + R[0] + R               for L, R in splits if R]
    if len(known(doubling)) != 0:
        return doubling
    else:
        for d1 in doubling:
            splitsd1     = [(d1[:i], d1[i:])    for i in range(len(d1) + 1)]
            doubling2   = [L + R[0] + R               for L, R in splitsd1 if R]
        return doubling2

def Priority_2(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    "Priority 2: insert letter (ex. juce -> juice)"
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    inserts    = [L + c + R               for L, R in splits for c in letters]
    if len(known(inserts)) != 0:
        return inserts
    else:
        for d1 in inserts:
            splitsd1     = [(d1[:i], d1[i:])    for i in range(len(d1) + 1)]
            inserts2    = [L + c + R               for L, R in splits for c in letters]
        return inserts2

In [11]:
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(WORDS) == 32192
    assert sum(WORDS.values()) == 1115504
    assert WORDS.most_common(10) == [
     ('the', 79808),
     ('of', 40024),
     ('and', 38311),
     ('to', 28765),
     ('in', 22020),
     ('a', 21124),
     ('that', 12512),
     ('he', 12401),
     ('was', 11410),
     ('it', 10681)]
    assert WORDS['the'] == 79808
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.perf_counter()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.perf_counter() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [12]:
spelltest(Testset(open('spell-testset1.txt')))

correction(contende) => contended (9); expected contented (13)
correction(proplen) => people (891); expected problem (71)
correction(guic) => guns (111); expected juice (5)
correction(jucie) => julie (71); expected juice (5)
correction(juise) => guise (8); expected juice (5)
correction(juse) => just (767); expected juice (5)
correction(compair) => company (190); expected compare (29)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); expected minuscule (0)
correction(poartry) => party (298); expected poetry (10)
correction(stanerdizing) => stanerdizing (0); expected standardizing (0)
correction(biscutes) => disputes (27); expected biscuits (8)
correction(receite) => receive (95); expected receipt (13)
correction(reciet) => recite (4); expected receipt (13)
correction(remined) => remained (231); expected remind (9)
correction(annt) => anna (294); expected aunt (52)
correction(vistid) => viscid (3); expected visited